In [ ]:
from main import *
from quantum.Quantum import QuantumRegressor
import pennylane as qml
import numpy as np
from pennylane import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize, basinhopping
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_runtime.fake_provider import FakeCairoV2
from mitiq.zne.scaling import fold_global
from mitiq.zne.inference import RichardsonFactory, LinearFactory
import joblib
import mthree
import os
import json
import time

# from sklearn.utils.testing import ignore_warnings
# from sklearn.exceptions import InconsistentVersionWarning
# @ignore_warnings(category=InconsistentVersionWarning)

In [ ]:
os.getcwd()

In [ ]:
# save_path = './MITIQ_ZNE_Richardson/IQP_Full-Pauli-CRX/'
save_path = './IQP_Full-Pauli-CRX/'
# settings = 'IQP_Full-Pauli-CRX/IQP_Full-Pauli-CRX.json'
settings = './IQP_Full-Pauli-CRX/IQP_Full-Pauli-CRX.json'


# train_set = 'linear_train.bin'
# test_set = 'linear_test.bin'
# scalerpath = 'linear_scaler.bin'

title=None
# train_set = '5_DDCC_train.bin'
# test_set = '5_DDCC_test.bin'
# scalerpath = '5_DDCC_scaler.bin'

train_set = 'PCA5_Morgan_train.bin'
test_set = 'PCA5_Morgan_test.bin'
scalerpath = 'PCA5_Morgan_scaler.bin'

batch_size = 25

In [ ]:
X_train, y_train = load_dataset(train_set)
parse_settings(settings)
if DEVICE == 'qiskit.ibmq':
    save_token(instance, token)

global NUM_QUBITS
global X_DIM
if NUM_QUBITS is not None:
    X_DIM = NUM_QUBITS
elif X_DIM == 1:  # if X_DIM is None and num_qubits wasn't specified anywhere use a default value of 2.
    NUM_QUBITS = 2
    X_DIM = NUM_QUBITS

kwargs = create_kwargs()

if title is None:
    title = os.path.basename(settings)
    title, _ = os.path.splitext(title)

if test_set is not None:
    X_test, y_test = load_dataset(test_set)
else:
    X_test, y_test = None, None


with open(scalerpath,'rb') as f:
    scaler = joblib.load(f)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)




In [ ]:
print(f'Training model with dataset {train_set} \n at time {time.asctime()}... ')
st = time.time()


model = QuantumRegressor(**kwargs)
model.fit(X_train, y_train)
hyperparams = None

et = time.time()
print(f'Training complete taking {et - st} total seconds. ')



In [ ]:
# removes temporary file created during training.
if os.path.exists(title + '_tentative_model.bin'):
    os.remove('tentative_model.bin')
elif os.path.exists('tentative_model.bin'):
    os.remove('tentative_model.bin')

print(y_train.shape)

scores, test_pred, train_pred = evaluate(model, X_train, y_train, X_test, y_test, plot=True, title=title,y_scaler=scaler)

y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

name = title + '_predicted_values.csv'
train_pred, y_train, test_pred, y_test = train_pred.tolist(), y_train.tolist(), test_pred.tolist(), y_test.tolist()
df_train = pd.DataFrame({'Predicted': train_pred, 'Reference': y_train})
df_train['Data'] = 'Train'
df_test = pd.DataFrame({'Predicted': test_pred, 'Reference': y_test})
df_test['Data'] = 'Test'
df = pd.concat([df_train, df_test], ignore_index=True)
df = df[['Data', 'Predicted', 'Reference']]

df.to_csv(name, index=False)
print(f'Saved predicted values as {name}')

print(f'Model scores: {scores}. ')

results = scores


results['hyperparameters'] = hyperparams
results_title = title + '_results.json'
with open(results_title, 'w') as outfile:
    json.dump(results, outfile)
    pass
print(f'Saved model results as {results_title}. ')